# **Performing Preprocessing & EDA on the given data**

In [1]:
#Code For Preprocessing of The Network Data

import pandas as pd

og_dataset = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/Impression_Network_Dataset_CS101_2024.xlsx")
new_dataset = pd.DataFrame()

In [2]:
#Making Everything to uppercase so that uniformity remains in code ##exception was found at '2020mcb1225', '2020MCB1225'
og_dataset = og_dataset.applymap(lambda x: x.upper() if isinstance(x, str) else x)

# Extract roll numbers from email addresses
new_dataset['Roll Number'] = og_dataset['Email Address'].str.split('@').str[0]	#Extraction based on string seperated by ('@')


# Extract roll numbers from Impression columns
for i in range(1, 31):
    column_name = f'Your Impression {i}'  # Construct column name
    new_dataset[column_name] = og_dataset[column_name].str.split(' ').str[-1]   #Extraction based on last entity of string seperated by (' ') space


In [3]:
# Create an empty dictionary to store roll numbers and names
roll_name_map = {}

# Iterate over each column
for col in og_dataset.columns[2:]:
    # Iterate over each row in the column
    for impression in og_dataset[col]:
        if pd.notna(impression):  # Check if impression is not NaN
            parts = list(impression.split(' '))  # Split the impression into parts
            name = ' '.join(parts[:-1])  # Join all parts except the last one to get the name
            roll_number = parts[-1]  # Get the last part as roll number
            roll_name_map[roll_number] = name  # Update the roll_name_map dictionary


missing_roll_numbers = []

# Iterate over keys in roll_name_map
for roll_number in roll_name_map.keys():
    # Check if the roll number is not already in the 'Roll Number' column
    if roll_number not in new_dataset['Roll Number'].values:
        # Append the roll number to the list
        missing_roll_numbers.append(roll_number)

# Create a DataFrame for missing roll numbers
missing_roll_numbers_df = pd.DataFrame({'Roll Number': missing_roll_numbers})

# Concatenate the original DataFrame with the DataFrame containing missing roll numbers
new_dataset = pd.concat([new_dataset, missing_roll_numbers_df], ignore_index=True)


# Map roll numbers to names and add a new "Name" column
new_dataset['Name'] = new_dataset['Roll Number'].map(roll_name_map)

In [4]:
# Move "Name" column to the first position
name_column = new_dataset['Name']  # Assuming you have a "Name" column
new_dataset.drop(labels=['Name'], axis=1, inplace=True)  # Remove "Name" column from DataFrame
new_dataset.insert(0, 'Name', name_column)  # Insert "Name" column at the first position

In [5]:
#Remove (Make '0') all those entries of friend where person has written his/her own roll number
# Iterate through each unique roll number in the 'Roll Number' column
for index, row in new_dataset.iterrows():
  roll_number = new_dataset['Roll Number'][index]
  for column, value in row.iloc[2:].items():  # Iterate over items in each row starting from the third column
    if(value==roll_number):
      # Set the value at position (index, column) to zero
      new_dataset.loc[index, column] = 0

In [6]:
# Replace NaN values with 0
new_dataset = new_dataset.fillna(0)

In [7]:
# Export DataFrame to Excel
new_dataset.to_excel('/content/drive/MyDrive/Colab Notebooks/Modified Network Dataset.xlsx', index=False)

# This **Modified Network Dataset.xlsx** is used in Task1 , Task2 & Task3 .